In [1]:
import pandas as pd
import umap
from sentence_transformers import SentenceTransformer, util
import torch
import hdbscan
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
import time
%matplotlib inline

In [2]:
# df_text = pd.read_csv('cleaned_data.csv', usecols=['cleaned_description','lemmatized_description'], converters={'lemmatized_description': eval})
df_text = pd.read_csv('cleaned_reviews2.csv', usecols=['text','summary'])
df_text.head(5)

,text,summary
0,saltwater taffy great flavors soft chewy candy...,great just as good as the expensive brands
1,know cactus tequila unique combination ingredi...,the best hot sauce in the world
2,one boys needed lose weight put food floor chu...,my cats love this diet food better than their ...
3,cats happily eating felidae platinum two years...,my cats are not fans of the new food
4,daughter loves twizzlers shipment six pounds r...,lots of twizzlers just what you expect


In [4]:
summary = df_text['text'].to_list()

corpus, queries = train_test_split(summary, train_size=5000, test_size=50)
print(f"Train:{len(corpus)} Validate: {len(queries)}")

Train:5000 Validate: 50


In [33]:
def similarity(query_embeddings,docs_embeddings,max_n=10, top_k=5):
    cos_scores = util.pytorch_cos_sim(query_embeddings, docs_embeddings)[0]
    cos_scores = cos_scores.cpu()
    top_results = torch.topk(cos_scores, k=max_n)
    return zip(top_results[0], top_results[1])

In [34]:
def get_query_top_k(query, query_embeddings, docs, docs_embeddings, max_n=10, top_k=5, min_p=0.7, exact_match=True):
    count=0
    top_k_list = []
    for score, idx in similarity(query_embeddings, docs_embeddings, max_n=max_n, top_k=top_k):
        score = score.item()
        if count<top_k and ((score>min_p and exact_match) or (score<=0.99 and score>min_p)): # we skip exact match if so required, because of floating point precision we set exact match to 0.99
            count=count+1
            top_k_list.append({"query":query,"sentence":docs[idx],"score":score})
    return top_k_list

In [14]:
def sent_similarity(corpus, query, threshold=0.8, exact_match=True):
    start = time.time()
    embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    n = 10
    top_k = 5
    
    data = []
    for query in queries:
        query_embeddings = embedder.encode(query, convert_to_tensor=True)
        top_k_list = get_query_top_k(query, query_embeddings, corpus, corpus_embeddings, max_n = n, top_k = top_k, min_p=threshold, exact_match=exact_match)
        data.extend(top_k_list)
    
    df = pd.DataFrame(data)
#     df.to_csv(output_file,index=False,sep="\t")
    end = time.time()
    ex_time = int(end - start)
    print(f'execution time {ex_time}')
    return df, embedder, corpus_embeddings

In [15]:
df, embedder, corpus_embeddings = sent_similarity(corpus, queries, 0.8, True)

execution time 21


In [16]:
pd.set_option('display.max_colwidth', None)
df

,query,sentence,score
0,could get fact opened first filled halfway water cans small shrimp companys time drain least empty really packed top tiny cleaned shrimp taste great make really great salad addition little lettuce onions celery mayo course price little since get cans case amazon sale worth buying highly recommend,review refers amy organic soup low sodium minestronei always searching something tasty low sodium foods started low sodium several years back husbands doctor insisted added everyone low sodium quite adjustment first salt definitely th food group adjust thing trying find new different things could find neighborhood market amy low sodium minestrone taste bland used low sodium products doctor maybe little pepper may enjoy much already low sodium happy find something new flavorful buying campbell healthy request minestrone little sodium difference quite noticeable texture amy watery like home made veggies seem like real veggies campbell packed veggies good portion pasta well little bread makes great lunch servings per servings pretty generous size benefits soup beside low sodium really nutritious made al vegan organic ingredients low fat cholesterol heat serve good source vitamins iron benefit well unlike condensed soups water added also marked kosher ingredients filtered water organic tomatoes organic onions organic carrots organic kidney beans organic potatoes organic celery organic pasta organic green beans organic peas organic leeks pure herbs spices organic high oleic safflower sunflower oil sea salt organic garlic contain wheat facility also processes milk soy tree nuts seed products however use peanuts fish shellfish eggs sodium per serving mg dietary fiber gm sugar protein total fat way order shipped arrived promptly remember breathe eat healthy enjoy laurie,0.802478
1,best pancake mix celiac children like food usually begg taste things spit made chocolate chip cookies mix came home school even told said could tell made pancakes ate gone far best tasting mix,tried recommended store employee favorite gf pancake mix bought ever since gluten free new family daughter diagnosed allergy gluten since hunt works us sadly lot products pancake waffle mix however enthusiastic hit large family even skeptical non gf members family love make crepes truly come perfect every time mix also bolstered lot additional nutrition find gf mixes perfect sweetness batter considering even need syrup additional toppings,0.819380
2,high quality pure marzipan grade product need candy coating sugar top flavor rich best way eat small bites end eating four loaf one sitting big hurry love marzipan owe least give one try odds ordering regular basis sometimes amazon prime get get much though reading nutritional information might give heart palpitations,taste newton fruit thins pretty good nice crunchy treat really enjoyed cranberry citrus oat flavor like really thin crisp designer oatmeal cookie would liked product cookies smaller size snacks rather large round shape thin easily break crumble current size could easily consume one two bites eliminate crumbs packaging leaves lot desired outer foil bag bad inside plastic tray clumsy tough slide back resealable wrap combined large crumbly nature cookie already mentioned doubt much anyone buying store ends half cookies actually whole time get home ordering amazon easily guarantees half product inside pieces like said smaller size makes em easier eat,0.804243
3,high quality pure marzipan grade product need candy coating sugar top flavor rich best way eat small bites end eating four loaf one sitting big hurry love marzipan owe least give one try odds ordering regular basis sometimes amazon prime get get much though reading nutritional information might give heart palpitations,huge fan chocolate covered pretzels ms favorite candy saw product pretty much melted puddle giddy anticipation pretzel good crunchy sweet light low fat large ms grams fat problem largely made pretzel tiny barely coating chocola

In [17]:
# Get top 5 for random query

In [38]:
def get_random_sim(query, embedder, corpus_embeddings):
    query_embeddings = embedder.encode(query, convert_to_tensor=True)
    top_k_list = get_query_top_k(query, query_embeddings, corpus, corpus_embeddings,
                                 max_n = 10, top_k = 5, min_p=0.65, exact_match=True)
    return pd.DataFrame(top_k_list)

In [40]:
get_random_sim(summary[1201], embedder, corpus_embeddings)

,query,sentence,score
0,like love black licorice seriously recommend product eat love black licorice recently found observed reported nuitionists well evidence black licorice aid constipation comes flavors medicinal effects come black,write review raspberry licorice completing review original black variety starters craving something fruity chewy love sweet slightly tangy flavor raspberries licorice well soft texture easily chew satisfying fat free guilt keep reach high shelf consume,0.779617
1,like love black licorice seriously recommend product eat love black licorice recently found observed reported nuitionists well evidence black licorice aid constipation comes flavors medicinal effects come black,found good substitutions gluten filled foods exception licorice love although expensive,0.763085
2,like love black licorice seriously recommend product eat love black licorice recently found observed reported nuitionists well evidence black licorice aid constipation comes flavors medicinal effects come black,find drink anything special terms taste black cherry juice infused carbonation gave odd taste black cherries favorite fruit however tried drink make positive impression juice quite acidic feel fruit juice drink comes cans rather glass bottle would helped drink maintain juicy taste longer preservatives gluten free sugar artificial flavors drink special features stand well market crowded many drinks minute maid mixed berry juice fl oz bottles fl oz cal valls certified organic juices clementine igourmet com tang sugar free orange drink mix ounce units crystal light classic orange ounce unit nesbitts california variety pack,0.744290
3,like love black licorice seriously recommend product eat love black licorice recently found observed reported nuitionists well evidence black licorice aid constipation comes flavors medicinal effects come black,wonderful tea really supresses appetite love packaging also would highly recommend weight loss tea tried many claim one garcinia actually works supress appetite great tasting also tasted many could stand,0.741829
4,like love black licorice seriously recommend product eat love black licorice recently found observed reported nuitionists well evidence black licorice aid constipation comes flavors medicinal effects come black,surprised find black tea licorice usually licorice reserved herbal teas mix ginseng peppermint licorice unusual taste treat one would recommend others mint strong licorice sweet normally like ginseng teas ginseng stays back brew noticeable yet sure still get health benefits,0.734146


In [43]:
get_random_sim(summary[18774], embedder, corpus_embeddings)

,query,sentence,score
0,opened first box gold kili ginger brewing bags took sniff wow strong scent ginger told treat unsweetened brewing bags make fabulous ginger tea whatever else want add kick ginger like best ginger lingering afterburn may repeat later burp excellent product ginger lover,always hunt ginger tea spicy punch ginger teas overwhelmed flavors ingredients one however pure ginger little bit honey heaven like double extra kick delicious drink cups day highly recommend product like ginger time brew ginger tea scratch,0.897244
1,opened first box gold kili ginger brewing bags took sniff wow strong scent ginger told treat unsweetened brewing bags make fabulous ginger tea whatever else want add kick ginger like best ginger lingering afterburn may repeat later burp excellent product ginger lover,tasty spicy beverage may cup tea speak smell ginger strong opening pouch taste rather strong piquant many people used strong ginger flavor may putting even unpleasant idea ginger stops canada dry ginger ale probably like product grew ginger foods love closest thing compare indonesian ting ting jehe chewy ginger candy basically liquid version candy sweet spicy tickles throat going juiced fresh ginger root tastes like ginger infused juice made pungency diminished manufacturing served hot perfect sick often make ginger tea honey weather convenient way tasty hot beverage ginger honey used thousands years throughout world health medicinal benefits lemon flavor could detect however,0.873972
2,opened first box gold kili ginger brewing bags took sniff wow strong scent ginger told treat unsweetened brewing bags make fabulous ginger tea whatever else want add kick ginger like best ginger lingering afterburn may repeat later burp excellent product ginger lover,new runa taste consistent great balance ginger citrus kick also side effects somewhat get teas made great summer tea well,0.824901
3,opened first box gold kili ginger brewing bags took sniff wow strong scent ginger told treat unsweetened brewing bags make fabulous ginger tea whatever else want add kick ginger like best ginger lingering afterburn may repeat later burp excellent product ginger lover,really nice aroma opened packet mixture lemon ginger taste lemon particularly hold mouth moment ginger taste really prevalent flavor kick get eat fresh pickled ginger enjoy drink weird unpleasant aftertaste would like something like actual product probably would make grocery cart,0.803757
4,opened first box gold kili ginger brewing bags took sniff wow strong scent ginger told treat unsweetened brewing bags make fabulous ginger tea whatever else want add kick ginger like best ginger lingering afterburn may repeat later burp excellent product ginger lover,galanga galangal root spice similar ginger flavor appearance flavor different complexity intensity ginger root much robust woody ginger whereas easy slice ginger eat root something would eat outright recommendation order warm hot season without expedited shipping stars product arrived small amount decay mold root two large rootstocks package sealed plastic bag would benefited ventilation sealed condensation formed wetness constant contact root combined warmth spring caused small amount decay mold occur small amount removed without much difficulty one root cooked growing garden recommend using something akin bone saw cut rootstock quite woody,0.785466
